In [6]:
# JUPYTER TEST HARNESS FOR ConfigReader (with proper import fix)

import os, sys, json, shutil, textwrap, traceback
from pathlib import Path
from contextlib import contextmanager
from tempfile import TemporaryDirectory
import yaml

# --------------- Helper: temporary chdir ---------------
@contextmanager
def pushd(new_dir: Path):
    old_dir = Path.cwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(old_dir)

# --------------- Helper: YAML writer ---------------
def write_yaml(path: Path, data: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        yaml.safe_dump(data, f, sort_keys=False)

# --------------- Baseline "valid" configs ---------------
def baseline_configs():
    CONFIG_DATA = {
        "split": {
            "test_size": 0.2,
            "val_size": 0.2,
            "cv_splits": 3,
            "seed": 42,
            "row_limit": 1000,  # per dataset, pre-split
        },
        "paths": {
            "pd_dir": "data/raw/pd",
            "lgd_dir": "data/raw/lgd",
        },
        "dataset_pd": {
            "01_gmsc": True,
            "02_taiwan_creditcard": False,
        },
        "dataset_lgd": {
            "01_heloc": False,
            "03_loss2": False,
        },
    }

    CONFIG_EVAL = {
        "binary_threshold": 0.5,
        "round_digits": 5,
        "metrics": {
            "pd": {
                "accuracy": True,
                "brier": True,
                "f1": True,
                "precision": True,
                "recall": True,
                "aucroc": True,
                "aucpr": True,
                "h_measure": True,
            },
            "lgd": {
                "mse": True,
                "mae": True,
                "r2": True,
                "rmse": True,
            },
        },
    }

    CONFIG_EXPERIMENT = {
        "imbalance": False,
        "imbalance_ratio": 0.5,
        "categorical_encoding": "ordinal",
        "numerical_encoding": "quantile",
        "normalization": "standard",
        "num_nan_policy": "mean",
        "cat_nan_policy": "most_frequent",
        "max_epochs": 10,
        "batch_size": 32,
        "tune": False,
        "n_trials": 10,
    }

    CONFIG_METHOD = {
        "methods": {
            "pd": {
                "cb": False, "knn": False, "lgbm": False, "logreg": False,
                "nb": False, "rf": False, "svm": False, "xgb": False,
                "ncm": False, "dummy": False,
                "mlp": False, "tabnet": False, "tabpfn": True,
            },
            "lgd": {
                "cb": False, "knn": False, "lgbm": False, "lr": False, "rf": False, "xgb": False,
                "mlp": False, "tabnet": False, "tabpfn": False,
            },
        }
    }
    return CONFIG_DATA, CONFIG_EVAL, CONFIG_EXPERIMENT, CONFIG_METHOD

# --------------- Utility: build sandbox repo ---------------
def make_sandbox_repo(overrides=None, real_reader_path=None):
    """
    Creates a full fake repo with data/, config/, src/utils/, and copies your real config_reader.py
    """
    ov = overrides or {}
    tmp = TemporaryDirectory()
    root = Path(tmp.name)

    # create folder structure
    (root / "data/raw/pd").mkdir(parents=True, exist_ok=True)
    (root / "data/raw/lgd").mkdir(parents=True, exist_ok=True)
    cfg_dir = root / "config"
    src_utils = root / "src/utils"
    src_utils.mkdir(parents=True, exist_ok=True)

    # copy your actual config_reader.py
    if real_reader_path is None:
        raise RuntimeError("Please provide path to your real src/utils/config_reader.py file.")
    shutil.copy(real_reader_path, src_utils / "config_reader.py")

    # write YAML configs
    cdata, ceval, cexp, cmeth = baseline_configs()
    for cfg_name, patch in ov.items():
        if cfg_name == "CONFIG_DATA":
            cdata = patch(cdata)
        elif cfg_name == "CONFIG_EVALUATION":
            ceval = patch(ceval)
        elif cfg_name == "CONFIG_EXPERIMENT":
            cexp = patch(cexp)
        elif cfg_name == "CONFIG_METHOD":
            cmeth = patch(cmeth)

    write_yaml(cfg_dir / "CONFIG_DATA.yaml", cdata)
    write_yaml(cfg_dir / "CONFIG_EVALUATION.yaml", ceval)
    write_yaml(cfg_dir / "CONFIG_EXPERIMENT.yaml", cexp)
    write_yaml(cfg_dir / "CONFIG_METHOD.yaml", cmeth)

    return tmp, root, cfg_dir

# --------------- Import ConfigReader dynamically ---------------
def import_config_reader(repo_root: Path):
    src_path = repo_root / "src"
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    from utils.config_reader import ConfigReader  # noqa: E402
    return ConfigReader

# --------------- Run one test case ---------------
def run_case(title, real_reader_path, overrides=None, expect_ok=True):
    tmp, root, cfg_dir = make_sandbox_repo(overrides, real_reader_path)
    try:
        ConfigReader = import_config_reader(root)
        with pushd(root):
            print(f"\n=== {title} ===")
            if expect_ok:
                cfg = ConfigReader(config_dir=str(cfg_dir)).load().validate().to_dict()
                print("✅ Validation PASSED")
                print("Resulting merged dictionary:")
                print(json.dumps(cfg, indent=2))
            else:
                try:
                    _ = ConfigReader(config_dir=str(cfg_dir)).load().validate().to_dict()
                    print("❌ Expected failure, but validation PASSED")
                except Exception as e:
                    print("✅ Validation FAILED as expected")
                    print("Error message:")
                    print(textwrap.indent(str(e), prefix="  "))
    finally:
        tmp.cleanup()




In [8]:
# ==================== RUN TESTS ====================
from pprint import pprint

# Path to the real config_reader.py (works from notebooks/)
REAL_READER_PATH = (Path.cwd().parent / "src/utils/config_reader.py").resolve()

# Helper: better print for dictionaries
def pretty_print_dict(d: dict):
    print("\n🧩 Merged configuration dictionary:")
    pprint(d, width=100, sort_dicts=False)
    print("\n" + "=" * 80 + "\n")

# Override run_case to use pprint instead of json.dumps
def run_case(title, real_reader_path, overrides=None, expect_ok=True):
    tmp, root, cfg_dir = make_sandbox_repo(overrides, real_reader_path)
    try:
        ConfigReader = import_config_reader(root)
        with pushd(root):
            print(f"\n=== 🧪 TEST CASE: {title} ===")
            if expect_ok:
                cfg = ConfigReader(config_dir=str(cfg_dir)).load().validate().to_dict()
                print("✅ Validation PASSED")
                pretty_print_dict(cfg)
            else:
                try:
                    _ = ConfigReader(config_dir=str(cfg_dir)).load().validate().to_dict()
                    print("❌ Expected failure, but validation PASSED")
                except Exception as e:
                    print("✅ Validation FAILED as expected")
                    print("Error message:\n")
                    print(textwrap.indent(str(e), prefix="  "))
                    print("\n" + "=" * 80 + "\n")
    finally:
        tmp.cleanup()


# --------------------------------------------------------------------------------
# ✅ VALID CASES
# --------------------------------------------------------------------------------

run_case("VALID: PD only (baseline)", REAL_READER_PATH)

run_case(
    "VALID: LGD only (simple regression setup)",
    REAL_READER_PATH,
    overrides={
        "CONFIG_DATA": lambda d: (d.update({
            "dataset_pd": {k: False for k in d["dataset_pd"]},
            "dataset_lgd": {**{k: False for k in d["dataset_lgd"]}, "01_heloc": True},
        }) or d),
        "CONFIG_METHOD": lambda m: (m["methods"]["lgd"].update({"lr": True}) or m),
    },
)

run_case(
    "VALID: BOTH tasks enabled",
    REAL_READER_PATH,
    overrides={
        "CONFIG_DATA": lambda d: (d.update({
            "dataset_pd": {**d["dataset_pd"], "01_gmsc": True},
            "dataset_lgd": {**{k: False for k in d["dataset_lgd"]}, "01_heloc": True},
        }) or d),
        "CONFIG_METHOD": lambda m: (m["methods"]["lgd"].update({"lr": True}) or m),
    },
)

run_case(
    "VALID: row_limit None",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d["split"].update({"row_limit": None}) or d)},
)

run_case(
    "VALID: round_digits = 0 (edge case)",
    REAL_READER_PATH,
    overrides={"CONFIG_EVALUATION": lambda e: (e.update({"round_digits": 0}) or e)},
)

run_case(
    "VALID: PD with classical RF (and categorical_encoding != 'indices')",
    REAL_READER_PATH,
    overrides={
        "CONFIG_METHOD": lambda m: (m["methods"]["pd"].update({"tabpfn": False, "rf": True}) or m),
        "CONFIG_EXPERIMENT": lambda e: (e.update({"categorical_encoding": "onehot"}) or e),
    },
)


# --------------------------------------------------------------------------------
# ❌ INVALID CASES
# --------------------------------------------------------------------------------

run_case(
    "ERROR: no dataset selected",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d.update({
        "dataset_pd": {k: False for k in d["dataset_pd"]},
        "dataset_lgd": {k: False for k in d["dataset_lgd"]},
    }) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: test_size + val_size too large",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d["split"].update({"test_size": 0.5, "val_size": 0.2}) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: cv_splits < 2",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d["split"].update({"cv_splits": 1}) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: seed not integer",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d["split"].update({"seed": 'abc'}) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: row_limit below 100",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d["split"].update({"row_limit": 10}) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: folder paths missing",
    REAL_READER_PATH,
    overrides={"CONFIG_DATA": lambda d: (d.update({
        "paths": {"pd_dir": "no/such/pd", "lgd_dir": "no/such/lgd"},
    }) or d)},
    expect_ok=False,
)

run_case(
    "ERROR: binary_threshold outside [0,1]",
    REAL_READER_PATH,
    overrides={"CONFIG_EVALUATION": lambda e: (e.update({"binary_threshold": 2.0}) or e)},
    expect_ok=False,
)

run_case(
    "ERROR: PD dataset active but no PD metrics enabled",
    REAL_READER_PATH,
    overrides={"CONFIG_EVALUATION": lambda e: (
        e["metrics"]["pd"].update({k: False for k in e["metrics"]["pd"]}) or e)},
    expect_ok=False,
)

run_case(
    "ERROR: categorical_encoding invalid value",
    REAL_READER_PATH,
    overrides={"CONFIG_EXPERIMENT": lambda e: (e.update({"categorical_encoding": "invalid_option"}) or e)},
    expect_ok=False,
)

run_case(
    "ERROR: tune=True but n_trials < 1",
    REAL_READER_PATH,
    overrides={"CONFIG_EXPERIMENT": lambda e: (e.update({"tune": True, "n_trials": 0}) or e)},
    expect_ok=False,
)

run_case(
    "ERROR: PD dataset active but no PD method enabled",
    REAL_READER_PATH,
    overrides={"CONFIG_METHOD": lambda m: (m["methods"]["pd"].update({"tabpfn": False}) or m)},
    expect_ok=False,
)

run_case(
    "ERROR: LGD dataset active but no LGD method enabled",
    REAL_READER_PATH,
    overrides={
        "CONFIG_DATA": lambda d: (d.update({
            "dataset_pd": {k: False for k in d["dataset_pd"]},
            "dataset_lgd": {**{k: False for k in d["dataset_lgd"]}, "01_heloc": True},
        }) or d),
        "CONFIG_METHOD": lambda m: (m["methods"]["lgd"].update({"lr": False}) or m),
    },
    expect_ok=False,
)

run_case(
    "ERROR: Regression-only method (LR) used for PD",
    REAL_READER_PATH,
    overrides={"CONFIG_METHOD": lambda m: (m["methods"]["pd"].update({"tabpfn": False, "lr": True}) or m)},
    expect_ok=False,
)

run_case(
    "ERROR: Classification-only method (logreg) used for LGD",
    REAL_READER_PATH,
    overrides={
        "CONFIG_DATA": lambda d: (d.update({
            "dataset_pd": {k: False for k in d["dataset_pd"]},
            "dataset_lgd": {**{k: False for k in d["dataset_lgd"]}, "01_heloc": True},
        }) or d),
        "CONFIG_METHOD": lambda m: (m["methods"]["lgd"].update({"logreg": True}) or m),
    },
    expect_ok=False,
)

run_case(
    "ERROR: classical model + indices categorical encoding",
    REAL_READER_PATH,
    overrides={
        "CONFIG_METHOD": lambda m: (m["methods"]["pd"].update({"tabpfn": False, "rf": True}) or m),
        "CONFIG_EXPERIMENT": lambda e: (e.update({"categorical_encoding": "indices"}) or e),
    },
    expect_ok=False,
)



=== 🧪 TEST CASE: VALID: PD only (baseline) ===
✅ Validation PASSED

🧩 Merged configuration dictionary:
{'data': {'split': {'test_size': 0.2,
                    'val_size': 0.2,
                    'cv_splits': 3,
                    'seed': 42,
                    'row_limit': 1000},
          'paths': {'pd_dir': 'data/raw/pd', 'lgd_dir': 'data/raw/lgd'},
          'dataset_pd': {'01_gmsc': True, '02_taiwan_creditcard': False},
          'dataset_lgd': {'01_heloc': False, '03_loss2': False}},
 'experiment': {'imbalance': False,
                'imbalance_ratio': 0.5,
                'categorical_encoding': 'ordinal',
                'numerical_encoding': 'quantile',
                'normalization': 'standard',
                'num_nan_policy': 'mean',
                'cat_nan_policy': 'most_frequent',
                'max_epochs': 10,
                'batch_size': 32,
                'tune': False,
                'n_trials': 10},
 'evaluation': {'binary_threshold': 0.5,
            